# PopSim Data Preparation

## Prerequisites

A template named **"_prep0_geo_cross_walk.csv"** is already available in **"data"**. This is where the geocrosswalk data, e.g., from QGIS, should be inserted. The column names, meaning the names of the individual levels, can be freely chosen now but **cannot be renamed later**. An unlimited number of levels is possible. The example data can be deleted.

If you want to use census levels, use "ZENSUS100m" and enter all cell keys in the study area. Place "ZENSUS1km" and "ZENSUS10km" headers at the correct level (left to right), their cell keys will be added automatically.

From this, a template for the controls of the lowest level will be created.

## Configuration

**All settings are centralized here.** Edit these values before running the notebook.

In [ ]:
# =============================================================================
# USER CONFIGURATION - Edit these values
# =============================================================================

# Output settings
output_everything = False  # Set to True to output all intermediate PopSim files

# Geography settings
seed_geography = "STAAT"  # Should usually not be changed

# Household column - set this AFTER running Step 1 to see available columns
# This column from your control totals data will be used as the number of households
household_column = None  # e.g., "Insgesamt_Bevoelkerung" or "numberOfHouseholds"

# MiD (seed data) filtering settings
filter_mid = False  # If True, filters MiD by the criteria below
kernwo = 2          # Day of week: 1=Monday, 2=Tuesday-Thursday, 3=Friday, 4=Saturday-Sunday
regiostar17 = [121, 123, 124]  # Regional types - see https://bmdv.bund.de/SharedDocs/DE/Artikel/G/regionalstatistische-raumtypologie.html

# =============================================================================
# END USER CONFIGURATION
# =============================================================================

## Step 1: Initialize and Inspect Data

This step reads the geo crosswalk, creates the control totals template, and shows you the available columns so you can set `household_column` above.

In [ ]:
import json
import os
import pandas as pd
from unidecode import unidecode

print("Starting data preparation...")

# Read first cell from crosswalk CSV to determine lowest geography name
crosswalk_path = 'data/_prep0_geo_cross_walk.csv'
lowest_geography_name = unidecode(pd.read_csv(crosswalk_path, header=None).iloc[0, 0])

print(f"Lowest geography name: {lowest_geography_name}")

# Save updated (unidecoded) value back to the CSV
df_cross = pd.read_csv(crosswalk_path)
df_cross.rename(columns={df_cross.columns[0]: lowest_geography_name}, inplace=True)

# Auto-calculate ZENSUS1km from ZENSUS100m if the column exists
if "ZENSUS1km" in df_cross.columns and "ZENSUS100m" in df_cross.columns:
    df_cross["ZENSUS1km"] = df_cross["ZENSUS100m"].str.replace(
        r"100mN(\d{5})E(\d{5})",
        lambda m: f"1kmN{int(m.group(1))//10}E{int(m.group(2))//10}",
        regex=True
    )

df_cross.to_csv(crosswalk_path, index=False)

output_filename = f'data/_prep1_control_totals_{lowest_geography_name}.csv'

if not os.path.exists(output_filename):
    # Create empty template - user will fill in or provide their own data
    df_new = pd.DataFrame(columns=[lowest_geography_name])
    df_new.to_csv(output_filename, index=False)
    print(f"Created empty template '{output_filename}'")
    print("Please fill this file with your control totals data, then re-run this cell.")
else:
    print(f"'{output_filename}' already exists.")
    df_existing = pd.read_csv(output_filename)
    print(f"\n{'='*60}")
    print("AVAILABLE COLUMNS IN YOUR CONTROL TOTALS DATA:")
    print(f"{'='*60}")
    for i, col in enumerate(df_existing.columns):
        print(f"  {i+1}. {col}")
    print(f"{'='*60}")
    print("\nSet 'household_column' in the Configuration cell above to one of these columns.")
    print("This column should contain the number of households for each geographic unit.")

# Save vars to a config file
config = {
    "lowest_geography_name": lowest_geography_name,
    "output_everything": output_everything,
    "seed_geography": seed_geography,
    "filter_mid": filter_mid,
    "kernwo": kernwo,
    "regiostar17": regiostar17,
    "household_column": household_column,
}
with open("prep_config.json", "w") as config_file:
    json.dump(config, config_file)

print("\nConfiguration saved to prep_config.json")

## Step 2: Fill Control Totals

In the `_prep1_control_totals_{geography}.csv` file in the data folder, the control data of the lowest level (e.g., grid cell, building block, or even building) must be inserted.

**Important:** Make sure `household_column` is set in the Configuration cell above before proceeding.

## Step 3: Reconcile and Create Templates

It is possible that the raw data for the lowest level does not cover all blocks (or buildings, etc.) created in the geo-cross-walk, or that there are data for blocks that are not considered. This will be reconciled here.

Then, templates for the controls of all levels will be created.

In [ ]:
import pandas as pd
import numpy as np
import os
import json
from unidecode import unidecode

# Load vars
with open("prep_config.json", "r") as config_file:
    config = json.load(config_file)

lowest_geography_name = config["lowest_geography_name"]
household_column = config["household_column"]

# Validate household_column is set
if household_column is None:
    raise ValueError(
        "household_column is not set! Please set it in the Configuration cell above "
        "to specify which column contains the number of households."
    )

# Read the two CSV files into Pandas DataFrames
df1 = pd.read_csv('data/_prep0_geo_cross_walk.csv')
df2 = pd.read_csv(f'data/_prep1_control_totals_{lowest_geography_name}.csv')

# Validate household_column exists in control totals
if household_column not in df2.columns:
    raise ValueError(
        f"household_column '{household_column}' not found in control totals data. "
        f"Available columns: {list(df2.columns)}"
    )

# Rename household column to numberOfHouseholds for PopSim compatibility
if household_column != "numberOfHouseholds":
    df2 = df2.rename(columns={household_column: "numberOfHouseholds"})
    print(f"Renamed '{household_column}' to 'numberOfHouseholds' for PopSim compatibility.")

# Drop duplicates from each DataFrame
len1_before, len2_before = len(df1), len(df2)
df1 = df1.drop_duplicates(subset=[lowest_geography_name])
df2 = df2.drop_duplicates(subset=[lowest_geography_name])
len1_after, len2_after = len(df1), len(df2)
print(f"Removed {len1_before - len1_after} duplicates from geo_cross_walk")
print(f"Removed {len2_before - len2_after} duplicates from control_totals_{lowest_geography_name}")

# Extract the common values
common_values = set(df1[lowest_geography_name]).intersection(df2[lowest_geography_name])

# Filter DataFrames
filtered_df1 = df1[df1[lowest_geography_name].isin(common_values)]
filtered_df2 = df2[df2[lowest_geography_name].isin(common_values)]

# Sort both by lowest geography
filtered_df1 = filtered_df1.sort_values(by=lowest_geography_name).reset_index(drop=True)
filtered_df2 = filtered_df2.sort_values(by=lowest_geography_name).reset_index(drop=True)

print(f"Number of rows in _prep0_geo_cross_walk: {len(df1)}")
print(f"Number of rows in geo_cross_walk (i.e. given geo-cells that actually have values): {len(filtered_df1)}")
print(f"Removed {len(df1) - len(filtered_df1)} rows")

print(f"Number of rows in _prep1_control_totals_{lowest_geography_name}: {len(df2)}")
print(f"Number of rows in control_totals_{lowest_geography_name} (i.e. control_totals that apply to the given geo-cells): {len(filtered_df2)}")
print(f"Removed {len(df2) - len(filtered_df2)} rows")


def unidecode_and_clean_column_names(df):
    modified_columns = {}
    for col in df.columns:
        clean = unidecode(col).replace(" ", "").replace(".", "").replace(",", "")
        modified_columns[col] = clean
    df.rename(columns=modified_columns, inplace=True)


unidecode_and_clean_column_names(filtered_df1)
unidecode_and_clean_column_names(filtered_df2)

# Confirm keys match
if not np.array_equal(filtered_df1[lowest_geography_name].values, filtered_df2[lowest_geography_name].values):
    raise ValueError("Mismatch between keys in geo_cross_walk and control totals. Aborting.")

# Save cleaned geo_cross_walk
filtered_df1.to_csv('data/geo_cross_walk.csv', index=False)
print(f"Created data/geo_cross_walk.csv")

# Create control_total templates for all higher-level geographies
geo_names = list(filtered_df1.columns)
for i, geo_name in enumerate(geo_names):
    if i == 0:
        continue  # Skip lowest geography

    data = filtered_df1.iloc[:, i:].drop_duplicates(subset=geo_name)
    if geo_name == "ZENSUS1km":
        # Populate ZENSUS1km from census data if available
        census1km_path = "data/ZENSUS1km.csv"
        if os.path.exists(census1km_path):
            census1km_df = pd.read_csv(census1km_path)
            data = data.merge(census1km_df, how="left", left_on=geo_name, right_on="ZENSUS1km")
    output_filename = f'data/_prep2_control_totals_{geo_name}.csv'
    if not os.path.exists(output_filename):
        data.to_csv(output_filename, index=False)
        print(f"Created {output_filename}")
    else:
        print(f"{output_filename} already exists, not overwritten.")

# Create finished control_total for lowest geography
merged = pd.concat([filtered_df1, filtered_df2.iloc[:, 1:]], axis=1)

for col in merged.columns:
    if col not in geo_names:
        merged.rename(columns={col: f"{col}_{lowest_geography_name}"}, inplace=True)
merged = merged.fillna(0)
merged.to_csv(f'data/control_totals_{lowest_geography_name}.csv', index=False)
print(f"Created data/control_totals_{lowest_geography_name}.csv")

# Update config with geo names
config["geo_names"] = geo_names
with open("prep_config.json", "w") as config_file:
    json.dump(config, config_file)

## Step 4: Fill Higher-Level Controls

Now the `_prep2_*` files for the remaining geographies must be filled with data by the user (if using only census, just press play). The prepared data must not be overwritten.

The settings file is fitted, and the controls file prepared.

In [ ]:
import pandas as pd
import os
from unidecode import unidecode
import json
import yaml

# Load vars
with open("prep_config.json", "r") as config_file:
    config = json.load(config_file)

lowest_geography_name = config["lowest_geography_name"]
geo_names = config["geo_names"]
output_everything = config["output_everything"]
seed_geography = config["seed_geography"]


def unidecode_and_clean_column_names(df):
    modified_columns = {}
    for unclean_column_name in df.columns:
        modified_name = unidecode(unclean_column_name)
        modified_name = modified_name.replace(" ", "").replace(".", "").replace(",", "")
        modified_columns[unclean_column_name] = modified_name
    df.rename(columns=modified_columns, inplace=True)


# Open all control_total files
control_total_files = [file for file in os.listdir('data') if
                       file.startswith('_prep2_control_totals_') or file.startswith('_prep1_control_totals_')]

# Create a dictionary to store data for the new control structure
new_data = {
    'target': [],
    'geography': [],
    'seed_table': [],
    'importance': [],
    'control_field': [],
    'expression': []
}

# Ensure the number of files matches the geographies
if len(control_total_files) != len(geo_names):
    raise ValueError("The number of control_total files does not match the number of geographies. Aborting.")

total_hh_control = None

# Process each control_total file
for file in control_total_files:
    df = pd.read_csv(f'data/{file}')
    unidecode_and_clean_column_names(df)

    parts = file.split('_')
    if len(parts) > 3:
        geography_name = parts[4].split('.')[0]
        if geography_name != df.columns[0] or geography_name not in geo_names:
            raise ValueError("File or file name does not match the expected format. Aborting.")
    else:
        raise ValueError("File name does not match the expected format. Aborting.")

    for column_name in df.columns:
        if column_name not in geo_names:
            new_name = f'{column_name}_{geography_name}'
            df.rename(columns={column_name: new_name}, inplace=True)

            if new_name.startswith("numberOfHouseholds") and geography_name == lowest_geography_name:
                total_hh_control = f'{new_name}_target'

            new_data['target'].append(f'{new_name}_target')
            new_data['geography'].append(geography_name)
            new_data['control_field'].append(new_name)

    # Save as cleaned file if not already existing
    output_clean_name = f'data/{file.replace("_prep2_", "")}'
    if not os.path.exists(output_clean_name):
        df = df.fillna(0)
        df.to_csv(output_clean_name, index=False)
        print(f"Created {output_clean_name}")
    else:
        print(f"{output_clean_name} already exists, not overwritten.")

if total_hh_control is None:
    raise ValueError("Could not find a total_hh_control. Aborting.")

# Insert example expression
new_data['expression'].append('(households.H_GEW > 0) & (households.H_GEW < np.inf)')

# Convert to DataFrame and save
df = pd.DataFrame({key: pd.Series(value) for key, value in new_data.items()})
output_file_name = os.path.join('configs', '_prep3_controls.csv')
if not os.path.exists(output_file_name):
    df.to_csv(output_file_name, index=False)
    print(f"Created controls file template '{output_file_name}'.")
else:
    print(f"Controls file template '{output_file_name}' already exists, not overwritten.")

# Edit the YAML settings file ----------------------------------------------
popsim_settings_file_name = os.path.join('configs', 'settings.yaml')
with open(popsim_settings_file_name, "r") as yaml_file:
    popsim_settings = yaml.safe_load(yaml_file)

topdown_geos = geo_names[::-1]
popsim_settings["geographies"] = topdown_geos
popsim_settings["seed_geography"] = seed_geography
popsim_settings["total_hh_control"] = total_hh_control

# Trim input_table_list after geo_cross_walk
index_to_remove_from = None
for i, table in enumerate(popsim_settings["input_table_list"]):
    if table["tablename"] == "geo_cross_walk":
        index_to_remove_from = i

if index_to_remove_from is not None:
    popsim_settings["input_table_list"] = popsim_settings["input_table_list"][:index_to_remove_from + 1]
else:
    raise ValueError("Could not find 'geo_cross_walk' in the input_table_list. Aborting.")

# Add new control table entries
for geo_name in geo_names:
    tablename = f"{geo_name}_control_data"
    filename = f"control_totals_{geo_name}.csv"
    popsim_settings["input_table_list"].append({
        "tablename": tablename,
        "filename": filename
    })

# Output tables
if output_everything:
    popsim_settings["output_tables"]["action"] = "skip"
    popsim_settings["output_tables"]["tables"] = "geo_cross_walk"
else:
    popsim_settings["output_tables"]["action"] = "include"
    popsim_settings["output_tables"]["tables"] = ["expanded_household_ids"]
    for geo_name in geo_names:
        popsim_settings["output_tables"]["tables"].append(f"summary_{geo_name}")
    for geo_name in geo_names[:-2]:
        popsim_settings["output_tables"]["tables"].append(f"summary_{geo_name}_{seed_geography}")

# Models
popsim_settings["models"] = [m for m in popsim_settings["models"] if "sub_balancing" not in m]
integerize_index = popsim_settings["models"].index("integerize_final_seed_weights")
for geo_name in geo_names[:-2]:
    popsim_settings["models"].insert(integerize_index + 1, f"sub_balancing.geography={geo_name}")

with open(popsim_settings_file_name, "w") as yaml_file:
    yaml.dump(popsim_settings, yaml_file, default_flow_style=False)
print("Modified settings.yaml file.")

# Edit analysis settings file ----------------------------------------------
analysis_settings_file_name = os.path.join('scripts', 'verification.yaml')
with open(analysis_settings_file_name, "r") as yaml_file:
    analysis_settings = yaml.safe_load(yaml_file)

analysis_settings["group_geographies"] = topdown_geos
try:
    this_folder_name = os.path.basename(os.path.dirname(os.path.abspath(__file__)))
except Exception:
    this_folder_name = os.path.basename(os.getcwd())
analysis_settings["region"] = this_folder_name

analysis_settings["seed_cols"]["geog"] = seed_geography
analysis_settings["summaries"] = [f"output/final_summary_{geo}.csv" for geo in geo_names]
analysis_settings["summaries"] += [f"output/final_summary_{geo}_{seed_geography}.csv" for geo in geo_names[:-2]]

analysis_settings["aggregate_summaries"] = [
    {
        'name': control_field,
        'geography': geography,
        'control': f'{target}_control',
        'result': f'{target}_result'
    }
    for target, geography, control_field in zip(new_data['target'], new_data['geography'], new_data['control_field'])
]

with open(analysis_settings_file_name, "w") as yaml_file:
    yaml.dump(analysis_settings, yaml_file, default_flow_style=False)
print("Modified verification.yaml file.")

## Step 5: Edit Controls

The user now has to edit the `_prep3_controls.csv` file in the configs folder. Add expressions and importance; seed_table is added automatically. The script will then filter the seed files accordingly and check for implausibilities.

In [ ]:
import pandas as pd
import os
import json
import re
import math

# Load config
with open("prep_config.json", "r") as config_file:
    config = json.load(config_file)
filter_mid = config["filter_mid"]
kernwo = config["kernwo"]
regiostar17 = config["regiostar17"]

# Load controls
controls_file_name = os.path.join('configs', '_prep3_controls.csv')
controls_df = pd.read_csv(controls_file_name)

# Load seed data
seed_persons_df = pd.read_csv('data/MiD2017_Personen.csv', sep=";")
seed_households_df = pd.read_csv('data/MiD2017_Haushalte.csv', sep=";")

# Filter MiD according to specs
if filter_mid:
    seed_persons_df = seed_persons_df[
        (seed_persons_df['RegioStaR17'].isin(regiostar17))
    ]
    seed_households_df = seed_households_df[
        seed_households_df['RegioStaR17'].isin(regiostar17)
    ]

print(f'Columns of seed_persons: {seed_persons_df.columns}')
print(f'Columns of seed_households: {seed_households_df.columns}')

# Define essential columns
column_names_to_include = {'H_ID', 'H_GEW', 'HP_ID', 'P_ID', 'P_GEW'}
unique_column_names = column_names_to_include.copy()

# Extract column names from expressions
pattern = r'\.(?P<column_name>[^ ]+)'
for expression in controls_df['expression']:
    if pd.isna(expression) or (isinstance(expression, float) and math.isnan(expression)):
        raise ValueError(
            f"Empty or NaN expression encountered. Make sure to write expressions for all targets in {controls_file_name} and rerun. Aborting."
        )

    matches = re.finditer(pattern, expression)
    for match in matches:
        column_name = match.group('column_name')
        unique_column_names.add(column_name)

# Filter columns
seed_persons_df = seed_persons_df[list(unique_column_names.intersection(seed_persons_df.columns))]
seed_households_df = seed_households_df[list(unique_column_names.intersection(seed_households_df.columns))]

print(f'Columns of seed_persons after filtering: {seed_persons_df.columns}')
print(f'Columns of seed_households after filtering: {seed_households_df.columns}')

# Add constant geography
seed_persons_df['STAAT'] = 1
seed_households_df['STAAT'] = 1

# Save outputs
seed_persons_df.to_csv('data/seed_persons.csv', index=False)
print("Created data/seed_persons.csv")
seed_households_df.to_csv('data/seed_households.csv', index=False)
print("Created data/seed_households.csv")
controls_df.to_csv('configs/controls.csv', index=False)
print("Created configs/controls.csv")

## Step 6: Cleanup (Optional)

Clean up all unnecessary intermediate files that were created only for verification.

In [ ]:
import os


def delete_files(folders, prefix, extension):
    """Delete files matching prefix and extension in specified folders (not subfolders)."""
    for folder in folders:
        if os.path.exists(folder):
            files_in_folder = [file for file in os.listdir(folder) if
                               file.startswith(prefix) and file.endswith(extension)]
            for file in files_in_folder:
                file_path = os.path.join(folder, file)
                try:
                    os.remove(file_path)
                    print(f"Deleted: {file_path}")
                except OSError as e:
                    print(f"Error deleting {file_path}: {e}")
        else:
            print(f"Folder not found: {folder}")


folders_to_process = ['data', 'configs', 'configs_mp']

# Uncomment the lines below to delete intermediate files
# delete_files(folders_to_process, '_prep', '.csv')  # Delete all _prep*.csv files

## Running PopSim

Use in anaconda console:
```
conda activate popsim
run_populationsim.py
```